In [1]:
import os
import numpy as np
import pandas as pd
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
df = pd.read_csv('STONK.csv',index_col='Date',parse_dates=['Date'])
# df = df[(df['Name']=='AAPL')]
df = df.dropna()
df.head

<bound method NDFrame.head of              Open   High    Low  Close    Volume  Name
Date                                                  
2006-01-03  77.76  79.35  77.24  79.11   3117200   MMM
2006-01-04  79.49  79.49  78.25  78.71   2558000   MMM
2006-01-05  78.41  78.65  77.56  77.99   2529500   MMM
2006-01-06  78.64  78.90  77.64  78.63   2479500   MMM
2006-01-09  78.50  79.83  78.46  79.02   1845600   MMM
...           ...    ...    ...    ...       ...   ...
2017-12-22  71.42  71.87  71.22  71.58  10979165  AABA
2017-12-26  70.94  71.39  69.63  69.86   8542802  AABA
2017-12-27  69.77  70.49  69.69  70.06   6345124  AABA
2017-12-28  70.12  70.32  69.51  69.82   7556877  AABA
2017-12-29  69.79  70.13  69.43  69.85   6613070  AABA

[93587 rows x 6 columns]>

In [3]:
#Create a new dataframe with only the 'Close column
data = df.filter(['High'])
#Convert the dataframe to a numpy array
dataset = data.values

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.05983884],
       [0.0599549 ],
       [0.05925852],
       ...,
       [0.0524937 ],
       [0.05235277],
       [0.05219525]])

In [5]:
#Create the training data set
#Create the scaled training data set
train_data = scaled_data
#Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data), 60):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
        
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape

[array([0.05983884, 0.0599549 , 0.05925852, 0.05946578, 0.06023677,
       0.05955697, 0.05926681, 0.05891033, 0.05888546, 0.05840463,
       0.05812276, 0.05819737, 0.05781602, 0.05725229, 0.05673   ,
       0.05526263, 0.05493103, 0.0547818 , 0.05485641, 0.0547818 ,
       0.05490615, 0.05513828, 0.05382014, 0.05316521, 0.05315692,
       0.05314863, 0.05447506, 0.05456626, 0.05492274, 0.05550305,
       0.05555279, 0.0548647 , 0.05552792, 0.05548647, 0.05518802,
       0.05546989, 0.05553621, 0.05595072, 0.05546989, 0.05556108,
       0.05523776, 0.05481496, 0.05440874, 0.05402739, 0.05463258,
       0.05437558, 0.05440874, 0.05417661, 0.05519631, 0.05560253,
       0.05661394, 0.05660565, 0.0563901 , 0.05651446, 0.05718597,
       0.05785747, 0.05794038, 0.05808131, 0.05805644, 0.05933313])]
[0.05840462926117522]



In [6]:
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)
#Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape
assert not np.any(np.isnan(x_train))
assert not np.any(np.isnan(y_train))

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

#Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
1559/1559 [==============================] - 27s 17ms/step - loss: 6.5471e-04
Epoch 2/10
1559/1559 [==============================] - 26s 17ms/step - loss: 1.3945e-04
Epoch 3/10
1559/1559 [==============================] - 27s 17ms/step - loss: 3.7667e-04 0s - loss: 3.7744e- - ETA: 0s - loss: 3.7711e-0
Epoch 4/10
1559/1559 [==============================] - 29s 19ms/step - loss: 2.3303e-04
Epoch 5/10
1559/1559 [==============================] - 29s 18ms/step - loss: 2.5577e-04
Epoch 6/10
1559/1559 [==============================] - 28s 18ms/step - loss: 9.6650e-05
Epoch 7/10
1559/1559 [==============================] - 28s 18ms/step - loss: 1.5565e-04
Epoch 8/10
1559/1559 [==============================] - 28s 18ms/step - loss: 2.2281e-04
Epoch 9/10
1559/1559 [==============================] - 30s 19ms/step - loss: 1.6600e-05 4s - los - ETA: 2 - ETA: 1s
Epoch 10/10
1559/1559 [==============================] - 28s 18ms/step - loss: 2.1599e-04


In [8]:
model.save('StonkPredict.h5')

In [19]:
ydf = yf.Ticker("MMM").history(period="61d").High
ydf = ydf.dropna()
ydf

Date
2020-09-10    165.240563
2020-09-11    165.270296
2020-09-14    167.976880
2020-09-15    168.472602
2020-09-16    169.632573
                 ...    
2020-11-30    176.000000
2020-12-01    175.690002
2020-12-02    172.580002
2020-12-03    173.389999
2020-12-04    173.160004
Name: High, Length: 61, dtype: float64

In [20]:
dataset = ydf.values[:60]
dataset = dataset.reshape(-1, 1)
dataset.shape

(60, 1)

In [21]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data.shape

(60, 1)

In [22]:
x_pred = np.reshape(scaled_data, (1, scaled_data.shape[0], 1 ))

In [23]:
pred = model.predict(x_pred)
pred = scaler.inverse_transform(pred)
print(pred[0][0])

174.02971
